In [1]:
import ast
import os
import re
import json
import nltk
import mlflow
import pandas as pd
import tiktoken
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from omegaconf import OmegaConf
from openai import OpenAI
from flatten_dict import flatten

from src.prompts.ubertext import (
    MultiGrammarErrorCorrectionGenerationPrompt,
    GrammarErrorCorrectionAggregationPrompt
)
from src.utils.openai_batch_utils import retrieve_openai_batch
from src.utils.openai_batch_utils import submit_openai_batch
from src.utils.utils import get_multi_gec_correction_comparison_text
from src.utils.metrics import average_edit_distance
from src.utils.utils import normalize_spaces, generate_original_corrected_texts
from src.utils.comparison_to_html import save_comparison_to_html, save_ansi_to_html

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [3]:
load_dotenv("../../../.env")

True

In [5]:
ubertext_df = pd.read_csv("../../../datasets/ubertext_gec/raw_ubertext_gec.csv", index_col=0)
ubertext_df

,tokens,characters,words,sentences,sentence
0,512,1382,181,17,"🔴Оперативне зведення Генштабу станом на 00:00 18 березня. \nГоловне: 🔻Окупанти тимчасово позбавили Україну виходу до Азовського моря. \nНа підступах до Миколаєва ворога зупинили спільними діями підрозділів Сил оборони. \n🔻Через провал у просуванні військ окупанти завдають ракетно-бомбових ударів по мирному населенню. \nТільки протягом доби 18 березня окупанти нанесли 4 ракетних удари (14 ракет) та здійснили понад 40 авіанальотів. \n🔻Загарбники не полишають спроб відновити просування в бік Києва. \nВедуть повітряну розвідку дронами. \n🔻Продовжуються спроби блокування міст Суми та Харкова, а також намагання відновити наступ у напрямку Покровська. \nОкупанти посилили угруповання за рахунок введення з території рф двох підрозділів зі складу 6-ї загальновійськової армії та 1-ї танкової армії Західного військового округу. \n🔻На Луганському напрямку загарбники намагаються встановити контроль над населеними пунктами Рубіжне та Попасна, а також блокувати Сєверодонецьк. \n🔻В районі Рубіжного ворог закріпився на західних та північно-західних околицях міста, робить невдалі спроби здійснити вихід до південної частини міста. \nТривають бої за місто Попасна. \n🔻На Донецькому напрямку окупанти ведуть бойові дії в районах Верхньоторецького, Кримського, Авдіївки, Тарамчука. \nТриває блокада та спроби штурму Маріуполя. \nЗберігаємо спокій! \nРазом переможемо! \nСлава Україні! \n#stoprussia"
1,75,154,22,0,"💪🏻 Збройні Сили України впродовж минулої доби, 18 березня, знищили 12 ворожих повітряних цілей : ▫️2 літака ▫️3 вертольоти ▫️3 БпЛА ▫️4 крилаті ракети"
2,291,775,111,7,"Нацгвардійці захопили техніку загарбників на Луганщині ""Гарні новини на ніч. \nГвардійці захопили техніку окупантів на Луганщині. \nРосійські окупанти зазнали втрат під Кремінною. \nНацгвардія виявила дві бойові машини піхоти російських загарбників та знищила з гранатомета одну БМП-1"", — повідомив керівник Луганської ОВА Сергій Гайдай. \nВін розповів, що під час бойового зіткнення екіпаж іншої ворожої БМП-1 не витримав вогневого контакту та втік, покинувши зброю й техніку. \nНашим бійцям дісталася трофейна бойова машина піхоти БМП-1 у робочому стані з повним БК, придатна для виконання бойових завдань і ліквідації її колишніх господарів. \nПід час огляду залишеної документації в БМП-1 стало відомо, що в ньому перебував командир одного з підрозділів так званої ""ЛНР""."
3,27,78,10,0,"😄 Вірменія, Казахстан та Киргизстан вимагають від рф платитимито в доларах"
4,39,102,14,0,Міністри культури ЄСпросять ЮНЕСКО перенести 45 сесію Комітету всесвітньої спадщини з рф до Львова
...,...,...,...,...,...
719634,299,704,83,2,"🖕🇷🇺Авіація рф кружляє над Бердянськом 🇺🇦 🐥 @proberdiansk_b ot «Бавовна»уМелітополіта Токмаку пожвавила російських терористів по всій тимчасово окупованоній території України 8 січня 2023 р. транспортні та військові гелікоптери весь день летять у бік Бердянської коси, куди скоріш за все, в організовані заздалегідь шпиталі, звозять двухсотих. \n❗️Бердянці очікують і на посилення репресій з боку російських терористів Втім, рашисти з перших днів окупації і до сьогодні не припиняли русифікацію, тиск і залякування людей БердянсЬка 🐥@proberdiansk_bot #бердянск [ бердянськ]( #бердянськ) #бердянськцеукраїна [ мелітополь]( #мелітополь) #мелитополь [ #запоріжжя]( запоріжжя) #окупація #оккупация"
719635,109,292,45,4,"🔥Місцеві ЗМІ Мелітополя повідомляють що вже вдруге за останні пів години місто чує гучні вибухи. \nТакож у небо піднялась авіація рашистів. \nВсе це після того, як вночі довго горів та детонував склад БК рашистів, який вони влаштували на заводі Гідромаш. \nФото: нічної бавовни у Мелітополі."
719636,111,250,30,0,"🇺🇦«Ромео» – підприємець, любитель моря і рибалки, тепер – сапер Бердянського батальйону територіальної оборони, взяв зброю аби захистити свій край, та повернути жовто-блакитний прапор в рідне місто🇺🇦 🚀 НАПИСАТИ |ПІДПИСАТИСЬ | ПІДТРИМАТИ |РЕКЛАМА"
719637,0,2,0,0,


In [6]:
grammar_correction_prompt = MultiGrammarErrorCorrectionGenerationPrompt().prompt_template

print(grammar_correction_prompt)

input_variables=['num_corrections', 'original_text'] template='Виправте наступний текст, зробивши його граматично правильним.\nВиправте всі орфографічні, пунктуаційні, стилістичні, граматичні, лексичні та синтаксичні помилки.\nЯкщо помилок немає, повторіть оригінальний текст.\nЗгенеруйте {num_corrections} різні варіанти виправленого тексту з поясненнями.\nЕкрануй всі символи подвійних лапок у відповіді зворотною косою рискою\n\nФормат відповіді у JSON:\n[{{\n    "correction": "виправлений текст",\n    "explanation": "пояснення до виправлення"\n}}, ...]\n\nПриклади:\n\n1. Вхідний текст:\n   Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.\n\n   Виправлення:\n   [("Останні три місяці мого життя були надзвичайно насиченими подіями та емоціями, але нарешті у мене з’явилося кілька вільних годин та трохи енергії, щоб продовжити сері

In [7]:
import json


def prepare_jsonl_batches(
        df: pd.DataFrame,
        text_column: str,
        prompt_template: PromptTemplate,
        start_at: int,
        end_at: int,
        output_dir: str
):
    """
    Split the dataset into batches and save as JSONL files for batch processing.
    """
    filename = f"{output_dir}/ubertext_batched_multigec_input_{start_at}:{end_at}.jsonl"

    if os.path.exists(filename):
        os.remove(filename)

    for i in range(start_at, end_at + 1):
        row = df.loc[i]
        sample_id: int = str(i)
        sample_input_text: str = row[text_column]
        sample_request = {
            "custom_id": sample_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": parameters.multi_gec.model_name,
                "messages": [
                    {
                        "role": "user",
                        "content": prompt_template.format(
                            original_text=sample_input_text,
                            num_corrections=parameters.multi_gec.num_corrections
                        ),
                    }
                ],
                "temperature": parameters.multi_gec.temperature,
                "top_p": parameters.multi_gec.top_p,
            }
        }

        # Save the batch to a JSONL file
        with open(filename, "a") as f:
            f.write(json.dumps(sample_request) + "\n")

    print(f"Input request saved to: {filename}")

    return filename


for batch_id, (s_at, e_at) in parameters.processing.batches.multi_gec.items():
    # Prepare batches
    ubertext_df_subset = ubertext_df.loc[s_at:e_at]
    batch_filename = prepare_jsonl_batches(
        df=ubertext_df_subset,
        text_column="sentence",
        prompt_template=grammar_correction_prompt,
        start_at=s_at,
        end_at=e_at,
        output_dir=output_dir
    )
    batch_metadata = submit_openai_batch(
        client=client,
        batch_filename=batch_filename,
        start_at=s_at,
        end_at=e_at,
        description_prefix="ubertext-batched-multigec"
    )
    print(f"{batch_metadata.id}: [{s_at, e_at}]")

Input request saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_input_170001:175000.jsonl
Batch submitted for processing: batch_673f5fb5f7e481908d819ed9d2ff1a6e
batch_673f5fb5f7e481908d819ed9d2ff1a6e: [(170001, 175000)]
Input request saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_input_175001:180000.jsonl
Batch submitted for processing: batch_673f5fc8d1048190bf32cea6ce34e4e0
batch_673f5fc8d1048190bf32cea6ce34e4e0: [(175001, 180000)]
Input request saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_input_180001:185000.jsonl
Batch submitted for processing: batch_673f5fdb43b48190b3e99109b3edec54
batch_673f5fdb43b48190b3e99109b3edec54: [(180001, 185000)]
Input request saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_input_185001:190000.jsonl
Batch submitted for processing: batch_673f5fed9f2c8190a214f63f3fa77eac
batch_673f5fed9f2c8190a214f63f3fa77eac: [(185001, 190000)]
Input re

In [6]:
multi_gec_batches: str = "multi_gec:\n"
for batch_id, (s_at, e_at) in parameters.processing.batches.multi_gec.items():
    output_file = retrieve_openai_batch(
        client=client,
        batch_id=batch_id,
        output_dir=output_dir,
        start_at=s_at,
        end_at=e_at,
        description_prefix="ubertext_batched_multigec",
    )
    multi_gec_batches += f"\t{batch_id}: [{s_at}, {e_at}]\n"

    print(f"Batch {batch_id}: {s_at, e_at} processing output saved to: {output_file}")

print()
print(multi_gec_batches)

Batch processing output saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_output_50001:55000.jsonl
Batch batch_673f2c1f07188190b80d2cdf18471535: (50001, 55000) processing output saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_output_50001:55000.jsonl
Batch processing output saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_output_55001:60000.jsonl
Batch batch_673f2d684cf48190bd0bf1f8e68f3752: (55001, 60000) processing output saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_output_55001:60000.jsonl
Batch processing output saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_output_60001:65000.jsonl
Batch batch_673f2f432d508190829c54ca7ec5fd2d: (60001, 65000) processing output saved to: ../../../datasets/ubertext/batched.nosync/ubertext_batched_multigec_output_60001:65000.jsonl
Batch processing output saved to: ../../../datasets/ubertext/batched.nosy